In [5]:
import numpy as np
import pandas as pd

### Основные идеи:

1) Сильными признаками оказались временные признаки: возраст вагона, время, оставшееся до планового ремонта, время, оставшееся до истечения срока службы

2) Признаки, связанные с номером станции, на которой в последний раз был вагон, также внесло сильный вклад в решение. Мы связавыем это с тем, что с определенных станций вагоны отправляются в депо раньше остальных (например, потому что ремонт там стоит дешевле)

3) Статистки за последний месяц помогают определить интенсивность нагрузки на вагон и являются сильной фичей

4) В качестве модели использовать градиентный бустинг из библиотеки LightGBM



#### Считываем данные

In [7]:
test = pd.read_csv('target/y_predict.csv')
test.month = pd.to_datetime(test.month)

In [8]:
def load_dislok():
    dislok = pd.read_parquet('dislok_wagons.parquet').convert_dtypes()
    feb = pd.read_parquet('test/dislok_wagons.parquet').convert_dtypes()
    dislok = pd.concat([dislok, feb]).reset_index()
    dislok.plan_date = pd.to_datetime(dislok.plan_date)
    return dislok

def load_targs():
    train = pd.read_csv('target/y_train.csv')
    feb = pd.read_csv('test/target/y_test.csv')
    train = pd.concat([train, feb]).reset_index()
    return train

def load_pr_rems():
    pr_rem1 = pd.read_parquet('./pr_rems.parquet').convert_dtypes()
    pr_rem2 = pd.read_parquet('./test/pr_rems.parquet').convert_dtypes()
    return pd.concat([pr_rem1, pr_rem2])

def load_tr_rems():
    tr_rem1 = pd.read_parquet('./tr_rems.parquet').convert_dtypes()
    tr_rem2 = pd.read_parquet('./test/tr_rems.parquet').convert_dtypes()
    return pd.concat([tr_rem1, tr_rem2]).reset_index()

#### Функция Считает статистики за предыдущий месяц. Достаем инфу из dislok в словарь, разворачиваем на таргет

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin
from tqdm.notebook import tqdm as tqdm

def process_ost_prob(accum, load_accum):


    accum = np.array(accum)
    load_accum = np.array(load_accum)


    if(np.isnan(accum).any()):
        return np.array([-1, -1, -1, -1, -1, -1])

    mask = accum != 160000
    accum = accum[mask]
    lam = load_accum[mask]

    if(len(accum) == 0):
        return np.array([0, 0, 0, 0, 0, 0])

    path = 0

    val_acc = []

    under_load = 0

    for i in range(len(accum) - 1):
        if(accum[i] < accum[i + 1]):
            pass
        else:
            val_acc.append(accum[i] - accum[i + 1])
            if(lam[i] == 1):
                under_load += accum[i] - accum[i + 1]

    return np.array([sum(val_acc), np.mean(val_acc), np.std(val_acc), sum(load_accum), under_load, under_load/(sum(val_acc)+1e-7)])

def unfold_stats_applyer(d, x):
    m = x["month"]
    cm = int(m[5:7])
    wn = x["wagnum"]
    if((wn, cm - 1) in d):
        ans = d[(wn, (cm - 1)%12 )]
    else:
        ans = np.array([np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])

    return pd.Series(data=ans, index=["last_month_run", "last_month_mean", "last_month_std", "under_load_days", "under_load_dist", "under_load_percent"])


class DislokStatExctractor(BaseEstimator, TransformerMixin):
    def __init__(self, dislok):
        dislok_sorted = dislok.sort_values(["wagnum", "plan_date"])

        plandat = dislok_sorted.plan_date.dt.month.values.copy()
        wagons = dislok_sorted.wagnum.values.copy()
        ost_prob = dislok_sorted.ost_prob.values.copy()

        is_load = dislok_sorted.isload.values.copy()

        del dislok_sorted

        accum = []
        load_accum = []

        d = {}
        for i in tqdm(range(len(plandat) - 1)):
            cm = plandat[i]
            nm = plandat[i + 1]
            wn = wagons[i]
            nwn = wagons[i + 1]

            if(ost_prob[i] is pd.NA):
                accum.append(np.nan)
            else:
                accum.append(ost_prob[i])
            load_accum.append(is_load[i])

            if(cm == nm)and(wn == nwn):
                pass
            else:
                d[(wn, cm%12)] = process_ost_prob(accum, load_accum)
                accum = []
                load_accum = []

        self.d_stats = d

    def fit(self, X, y = None):
        return self

    def transform(self, X, y = None):
        return X.apply(lambda x: unfold_stats_applyer(self.d_stats, x), axis=1)

#### Время до ближайшего планового ремонта. Сохраняем все времена из таблицы по ключу вагона, затем для каждого вагона из taget ищим минимальную полжительную разницу в словаре.

In [10]:
class NearestRepair(BaseEstimator, TransformerMixin):
  def __init__(self, base_date_name="month", repair_date_name="date_pl_rem"):
    self.base_date_name = base_date_name
    self.repair_date_name = repair_date_name
    self.wagnum = "wagnum"
    self.bound_date = pd.to_datetime('2022-01-01')

  def fit(self, dislok):
    rep_dates = {}
    for idx, data in tqdm(dislok[[self.wagnum, self.repair_date_name]].groupby(self.wagnum)):
      rep_dates[idx] = np.array((pd.to_datetime(data[self.repair_date_name].unique()) - self.bound_date).total_seconds(), dtype=np.int64)
    self.rep_dates = rep_dates
    return self

  def transform(self, wagons):
    wagons["nearrepair"] = pd.Series((wagons[self.base_date_name] - self.bound_date).dt.total_seconds(), dtype=np.int64)
    wagons = wagons.reset_index()
    for i, row in tqdm(wagons.iterrows()):
        if row[self.wagnum] in self.rep_dates:
            seconds = self.rep_dates[row[self.wagnum]] - row["nearrepair"]
            seconds = seconds[seconds >= 0]
            if len(seconds):
                seconds = min(seconds)
            else:
                seconds = pd.NA
        else:
            seconds = pd.NA
        wagons.loc[i, "nearrepair"] = seconds
    return wagons

#### Далее мы обрабатываем таблицу Dislok, и добавляем фичи из нее, на какой станции остановился поезд в первое число каждого месяца, остановился ли он у станции, у которых часто происходят ремонтные работы, сколько пробега у него осталось на первое число каждого месяца и сколько у него времени осталось до планового ремонта, а также код ремонта

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin
from collections import Counter

class DislokTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, times, ids, stat_remont):
        dislok = pd.read_parquet('dislok_wagons.parquet').convert_dtypes()
        feb = pd.read_parquet('test/dislok_wagons.parquet').convert_dtypes()
        self.dislok = pd.concat([dislok, feb])
        self.dislok = self.dislok[['plan_date', 'wagnum', 'ost_prob', 'kod_vrab', 'date_pl_rem', 'isload', 'distance', 'st_id_send', 'st_id_dest']]
        self.dislok.plan_date = pd.to_datetime(self.dislok.plan_date)
        self.dislok.st_id_send = self.dislok.st_id_send.apply(hash)
        self.dislok.st_id_dest = self.dislok.st_id_dest.apply(hash)
        self.dislok = self.dislok.sort_values(by=['wagnum', 'plan_date']).reset_index()
        self.stat_remont = stat_remont
        self.times = times
        self.ids = ids
        self.wagon_probeg = {}
        self.wagon_kod_remont = {}
        self.time_remont = {}
        self.stat_remont = {}
        self.cur_st_send = {}
        self.cur_st_dest = {}

    def fit(self, train):
        index_train = 0
        for train_id in tqdm(self.ids):
            while self.dislok.loc[index_train, 'wagnum'] != train_id:
                index_train += 1
            self.wagon_kod_remont[train_id] = self.dislok.loc[index_train, 'kod_vrab']
            self.time_remont[train_id] = self.dislok.loc[index_train, 'date_pl_rem']
            for index, train_time in enumerate(self.times):
                while self.dislok.loc[index_train, 'plan_date'] < train_time and self.dislok.loc[index_train, 'wagnum'] == train_id:
                    index_train += 1
                if not index:
                    if self.dislok.loc[index_train, 'plan_date'] == train_time and self.dislok.loc[index_train, 'wagnum'] == train_id:
                        self.wagon_probeg[train_id] = [self.dislok.loc[index_train, 'ost_prob']]
                        self.stat_remont[train_id] = [self.dislok.loc[index_train, 'st_id_send'] in self.stat_remont or self.dislok.loc[index_train, 'st_id_dest'] in self.stat_remont]
                        self.cur_st_send[train_id] = [self.dislok.loc[index_train, 'st_id_send']]
                        self.cur_st_dest[train_id] = [self.dislok.loc[index_train, 'st_id_dest']]
                    else:
                        self.wagon_probeg[train_id] = [pd.NA]
                        self.stat_remont[train_id] = [pd.NA]
                        self.cur_st_send[train_id] = [pd.NA]
                        self.cur_st_dest[train_id] = [pd.NA]
                else:
                    if self.dislok.loc[index_train, 'plan_date'] == train_time and self.dislok.loc[index_train, 'wagnum'] == train_id:
                        self.wagon_probeg[train_id].append(self.dislok.loc[index_train, 'ost_prob'])
                        self.stat_remont[train_id].append(self.dislok.loc[index_train, 'st_id_send'] in self.stat_remont or self.dislok.loc[index_train, 'st_id_dest'] in self.stat_remont)
                        self.cur_st_send[train_id].append(self.dislok.loc[index_train, 'st_id_send'])
                        self.cur_st_dest[train_id].append(self.dislok.loc[index_train, 'st_id_dest'])
                    else:
                        self.wagon_probeg[train_id].append(pd.NA)
                        self.stat_remont[train_id].append(pd.NA)
                        self.cur_st_send[train_id].append(pd.NA)
                        self.cur_st_dest[train_id].append(pd.NA)
        return self

    def transform(self, test):
        test = test.copy()
        test['probeg'] = 0
        test['kod_remont'] = 0
        test['time_before_remont'] = 0
        test['stat_remont'] = 0
        test['cur_st_send'] = 0
        test['cur_st_dest'] = 0
        for i, row in tqdm(test.iterrows()):
            if row['month'].month >= 8:
                month = row['month'].month - 8
            else:
                month = row['month'].month + 4
            test.loc[i, 'probeg'] = self.wagon_probeg[row['wagnum']][month]
            test.loc[i, 'stat_remont'] = self.stat_remont[row['wagnum']][month]
            test.loc[i, 'cur_st_send'] = self.cur_st_send[row['wagnum']][month]
            test.loc[i, 'cur_st_dest'] = self.cur_st_dest[row['wagnum']][month]
            test.loc[i, 'kod_remont'] = self.wagon_kod_remont[row['wagnum']]
            test.loc[i, 'time_before_remont'] = (self.time_remont[row['wagnum']] - row['month']).days
        test.stat_remont = test.stat_remont.fillna(-1).astype(int)
        return test


#### Добавляем фичи возраста вагона (из фичи года постройки вагона), времени до истечения срока службы и количество ремонтов

In [12]:
class WagParamsTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.wag_params = pd.read_parquet('wag_params.parquet').convert_dtypes().sort_values(by=['wagnum']).reset_index()
        self.wag_params.date_build = pd.to_datetime(self.wag_params.date_build)
        self.wag_params.srok_sl = pd.to_datetime(self.wag_params.srok_sl)
        self.srok_sl = {}
        self.age = {}
        for i, row in self.wag_params.iterrows():
            self.srok_sl[i] = row['srok_sl']
            self.age[i] = row['date_build']

        tr_rems = pd.read_parquet('tr_rems.parquet').convert_dtypes()
        feb = pd.read_parquet('test/tr_rems.parquet').convert_dtypes()
        self.tr_rems = pd.concat([tr_rems, feb]).sort_values(by=['wagnum']).reset_index()
        self.tr_rems = self.tr_rems.groupby('wagnum', as_index= False).kod_vrab.count()
        self.tr_rems = dict(zip(self.tr_rems.wagnum.to_list(), self.tr_rems.kod_vrab.to_list()))

    def fit(self, train):
        return self

    def transform(self, test):
        test = test.copy()
        test['time_before_srok_sl'] = -10000
        test['age'] = -10000
        test['kod_vrab_tr_rems'] = -10000
        for i, row in test.iterrows():
            test.loc[i, 'time_before_srok_sl'] = (self.srok_sl[row['wagnum']] - row['month']).days
            test.loc[i, 'age'] = (row['month'] - self.age[row['wagnum']]).days
            if row['wagnum'] in self.tr_rems:
                test.loc[i, 'kod_vrab_tr_rems'] = self.tr_rems[row['wagnum']]
        return test

#### Проходим по owner_ship и извлекаем manage_type и rod_id(значение одно и тоже для всех записей относящихся к конкретному вагону). rod_id дяльше не участвует т.к берется из другой таблицы.

In [13]:

class OwnerShipExtracor(BaseEstimator, TransformerMixin):
    def __init__(self, wagprob):
        tables = {}

        for col in ["manage_type", "rod_id"]:
            d = {}

            for w, day in tqdm(zip(wag_prob.wagnum, wag_prob[col])):
                if(w in d):
                    if not(day in d[w]):
                        d[w].add(day)
                else:
                    d[w] = set([day])

            dd = {"wagnum":[], col:[]}

            for k, v in d.items():
                assert len(v) == 1
                for x in v:
                    break
                dd["wagnum"].append(k)
                dd[col].append(x)
            print(col)
            tables[col] = pd.DataFrame.from_dict(dd) #.rod_id.value_counts()

        tb = tables["manage_type"].copy()
        self.tables = (tb.merge(tables["rod_id"].copy(), how="left", on="wagnum")).copy()

    def fit(self, X, y = None):
        return self

    def transform(self, X, y = None):
        return (X.merge(self.tables.copy(), how="left", on="wagnum")).copy()


#### Время до ближайшего ремонта, который был. Принцип генерации точно такой же как и ближайшего запланированного, тролько теперь рассматриваем противоположенные по знаку разницы

In [14]:
class LastRepair(BaseEstimator, TransformerMixin):
  def __init__(self):
    self.bound_date = pd.to_datetime('2022-01-01')
    self.prev_repairs = {}

  #pr_rems
  def fit(self, train):
    rep_dates = {}
    for idx, data in tqdm(train[["wagnum", "rem_month"]].groupby("wagnum")):
      rep_dates[idx] = pd.Series((pd.to_datetime(data["rem_month"].unique()) - self.bound_date).total_seconds(), dtype=np.int64)
    self.rep_dates = rep_dates
    return self

  #target
  def transform(self, test):
    test["lastrepair"] = pd.Series((pd.to_datetime(test["month"]) - self.bound_date).dt.total_seconds(), dtype=np.int64)
    for i, row in tqdm(test.iterrows()):
        if row["wagnum"] in self.rep_dates:
            seconds = self.rep_dates[row["wagnum"]] - row["lastrepair"]
            seconds = -seconds[seconds <= 0]
            cnt_repairs = len(seconds)
            if len(seconds):
                seconds = min(seconds)
            else:
                seconds = -1
        else:
            seconds = -1
            cnt_repairs = 0
        test.loc[i, "cnt_repairs"] = cnt_repairs
        test.loc[i, "lastrepair"] = seconds
    return test

In [15]:
dislok = load_dislok()
targs = load_targs()

In [16]:
targs_copy = targs.copy()
#pd.to_datetime(dislok.plan_date)
targs_copy.month = pd.to_datetime(targs_copy.month)

In [17]:
test = pd.read_csv('target/y_predict.csv')
test.month = pd.to_datetime(test.month)
train_times = sorted(list(set(pd.concat([targs_copy, test])['month'].to_list())))
train_id = sorted(list(set(targs_copy['wagnum'].to_list())))
tr_rem = pd.read_parquet('tr_rems.parquet').convert_dtypes()
feb = pd.read_parquet('test/tr_rems.parquet').convert_dtypes()
tr_rem = pd.concat([tr_rem, feb]).sort_values(by=['wagnum']).reset_index()
stat_remont = Counter(tr_rem.st_id_send.to_list()).most_common(11)
stat_remont = [i[0] for i in stat_remont]
transformer = DislokTransformer(train_times, train_id, stat_remont)
transformer.fit(targs_copy)
new_train = transformer.transform(targs_copy)

  0%|          | 0/33977 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [18]:
ef_test = transformer.transform(test.copy())

0it [00:00, ?it/s]

In [19]:
ef_test = ef_test[["probeg", "kod_remont", "time_before_remont", "stat_remont", "cur_st_send", "cur_st_dest"]].copy()
ef_test.probeg = ef_test.probeg.fillna(-10000)

In [20]:
ef = new_train[["probeg", "kod_remont", "time_before_remont", "stat_remont", "cur_st_send", "cur_st_dest"]].copy()

In [21]:
ef.probeg = ef.probeg.fillna(-10000)

In [22]:
transformer = WagParamsTransformer()
eff = transformer.transform(new_train)
eff = eff[['time_before_srok_sl', 'age', 'kod_vrab_tr_rems']].copy()

In [23]:
cat_feat = np.array(['cur_st_dest', 'cur_st_send', 'stat_remont', 'kod_remont'])
for p in cat_feat:
    ef[p] = ef[p].astype('category')
eff['kod_vrab_tr_rems'] = eff['kod_vrab_tr_rems'].astype('category')

In [24]:
eff_test = transformer.transform(test.copy())

In [25]:
eff_test = eff_test[['time_before_srok_sl', 'age', 'kod_vrab_tr_rems']].copy()
for p in cat_feat:
    ef_test[p] = ef_test[p].astype('category')
eff_test['kod_vrab_tr_rems'] = eff_test['kod_vrab_tr_rems'].astype('category')

In [26]:
NR = NearestRepair()

In [27]:
NR.fit(dislok)

  0%|          | 0/33977 [00:00<?, ?it/s]

NearestRepair()

In [28]:
nerep = NR.transform(targs_copy)[["nearrepair"]]

0it [00:00, ?it/s]

In [29]:
nerep.nearrepair = nerep.nearrepair.fillna(-1)

In [30]:
nerep_test = NR.transform(test.copy())[["nearrepair"]]
nerep_test.nearrepair = nerep_test.nearrepair.fillna(-1)

0it [00:00, ?it/s]

In [31]:
dislok = pd.read_parquet('dislok_wagons.parquet').convert_dtypes()
feb = pd.read_parquet('test/dislok_wagons.parquet').convert_dtypes()
dislok = pd.concat([dislok, feb]).reset_index()
de = DislokStatExctractor(dislok)

  0%|          | 0/7228236 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [32]:
test = pd.read_csv('target/y_predict.csv')
#test.month = pd.to_datetime(test.month)

mff = de.fit_transform(targs)
mff_test = de.transform(test.copy())

In [33]:
test = pd.read_csv('target/y_predict.csv')
test.month = pd.to_datetime(test.month)

mff_test

,last_month_run,last_month_mean,last_month_std,under_load_days,under_load_dist,under_load_percent
0,10000.0,370.370370,380.690031,21.0,9815.0,0.981500
1,4241.0,157.074074,300.058010,11.0,4112.0,0.969583
2,4338.0,160.666667,262.711361,9.0,4019.0,0.926464
3,9939.0,368.111111,270.018426,14.0,5058.0,0.508904
4,4590.0,170.000000,227.684642,11.0,2524.0,0.549891
...,...,...,...,...,...,...
33702,4561.0,168.925926,232.270903,12.0,3049.0,0.668494
33703,3661.0,135.592593,228.671244,9.0,1345.0,0.367386
33704,3434.0,127.185185,220.613125,6.0,1960.0,0.570763
33705,5872.0,217.481481,268.502896,19.0,5626.0,0.958106


In [34]:

wag_prob = pd.read_parquet('./wagons_probeg_ownersip.parquet').convert_dtypes()

oe = OwnerShipExtracor(wag_prob)

oe.fit(targs)

own_feat = (oe.transform(targs)[["manage_type"]]).copy()

for p in ["manage_type"]:
    own_feat[p] = own_feat[p].astype('category')

0it [00:00, ?it/s]

manage_type


0it [00:00, ?it/s]

rod_id


In [35]:
own_feat_test = (oe.transform(test.copy())[["manage_type"]]).copy()
for p in ["manage_type"]:
    own_feat_test[p] = own_feat_test[p].astype('category')

In [36]:
own_feat_test

,manage_type
0,0
1,0
2,0
3,0
4,0
...,...
33702,0
33703,0
33704,0
33705,0


In [37]:

lastrep = LastRepair()

pr_rem = load_pr_rems()

lastrep.fit(pr_rem)

lastreptab = lastrep.transform(targs)[["lastrepair", "cnt_repairs"]]

  0%|          | 0/11982 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [38]:
lastreptab_test = lastrep.transform(test.copy())[["lastrepair", "cnt_repairs"]]

0it [00:00, ?it/s]

In [39]:
lastreptab_test

,lastrepair,cnt_repairs
0,8640000,1.0
1,7171200,1.0
2,7516800,1.0
3,-1,0.0
4,12614400,1.0
...,...,...
33702,-1,0.0
33703,14169600,1.0
33704,11577600,1.0
33705,15379200,1.0


#### Достаем из таблицы tr_rem количество ремонтов до нужного месяца. Подражали бэйзлайну не очень сильная фича.

In [40]:

def unfold_trem_applyer(rem_dict, x):
    m = pd.to_datetime(x["month"])
    wn = x["wagnum"]

    if(wn in rem_dict):
        ans = np.array([rem_dict[wn].searchsorted(m, side="left") ])
    else:
        ans = np.array([0])

    return pd.Series(data=ans, index=["num_small_repairs"])


class SmallRepairsExctractor(BaseEstimator, TransformerMixin):
    def __init__(self, df):
        df = df.sort_values(["rem_month"])
        rem_dict = {}

        for x in tqdm(df.groupby("wagnum")):
            wg, ddf = x
            rem_dict[wg] = ddf.rem_month
        self.rem_dict = rem_dict

    def fit(self, X, y = None):
        return self

    def transform(self, X, y = None):
        return X.apply(lambda x: unfold_trem_applyer(self.rem_dict, x), axis=1)

In [41]:
tr_rems = load_tr_rems()
sre = SmallRepairsExctractor(tr_rems)
trems_counts = sre.transform(targs)
trems_counts.num_small_repairs = trems_counts.num_small_repairs.astype('category')

  0%|          | 0/19621 [00:00<?, ?it/s]

In [42]:
trems_counts_test = sre.transform(test)
trems_counts_test.num_small_repairs = trems_counts_test.num_small_repairs.astype('category')

In [43]:
trems_counts_test

,num_small_repairs
0,3
1,2
2,2
3,2
4,0
...,...
33702,0
33703,0
33704,4
33705,0


#### Считаем статистики по грузам которые перевозил вагон до нужного месяца.

In [44]:
tqdm.pandas()

def get_counts(wag_groups, wagnum, month):
    count_names = {
        0: "zerostype",
        1: "onestype",
        2: "twostype"
    }
    wag_slice = wag_groups[wagnum]
    wag_slice = wag_slice[wag_slice.plan_date <= month]
    wag_slice_loads = wag_slice[wag_slice.isload == 1]
    counts = wag_slice_loads.fr_class.value_counts([0, 1, 2], dropna=True)
    if len(wag_slice):
        counts *= len(wag_slice_loads) / len(wag_slice)

    if len(wag_slice_loads):
        skoroport = float(wag_slice_loads.skoroport.sum(skipna=True)) / len(wag_slice_loads)
        naval = float(wag_slice_loads.naval.sum(skipna=True)) / len(wag_slice_loads)
        naliv = float(wag_slice_loads.naliv.sum(skipna=True)) / len(wag_slice_loads)
    else:
        skoroport = 0
        naval = 0
        naliv = 0
    data_list = list(counts.values) + [skoroport, naval, naliv]

    col_names = [count_names[idx] for idx in counts.index] + ["skoroport_frac", "naval_frac", "naliv_frac"]
    return pd.Series(data=data_list, index=col_names)

class CountsCargo(BaseEstimator, TransformerMixin):
  def __init__(self):
    self.wag_groups = {}

  #dislok, freq_info
  def fit(self, dislok, freq_info):
    dislok_cargo = dislok[["plan_date", "wagnum", "isload", "fr_id"]]
    freq_cargo = freq_info[["fr_id", "fr_class", "skoroport", "naval", "naliv"]]
    dislok_cargo = dislok_cargo.merge(freq_cargo, how="left", on="fr_id")
    for i, wag_slice in tqdm(dislok_cargo.groupby("wagnum")):
        self.wag_groups[wag_slice["wagnum"].iloc[0]] = wag_slice
    return self

  #target
  def transform(self, test):
    counts_df = test.progress_apply(lambda x: get_counts(self.wag_groups, x["wagnum"], x["month"]), axis=1)
    counts_df = pd.concat([test, counts_df], axis=1)
    counts_df[["naliv_frac", "naval_frac", "onestype", "skoroport_frac", "twostype", "zerostype"]] = \
      counts_df[["naliv_frac", "naval_frac", "onestype", "skoroport_frac", "twostype", "zerostype"]].fillna(0.0)
    return counts_df

In [45]:
dan_transformer = CountsCargo()
freq_info = pd.read_parquet('./freight_info.parquet').convert_dtypes()
dan_transformer.fit(dislok, freq_info)
danf = dan_transformer.transform(targs)
danf = danf[['naliv_frac', 'naval_frac', 'onestype', 'skoroport_frac', 'twostype', 'zerostype']]

  0%|          | 0/33977 [00:00<?, ?it/s]

  0%|          | 0/237561 [00:00<?, ?it/s]

In [46]:
danf_test = dan_transformer.transform(test)
danf_test = danf_test[['naliv_frac', 'naval_frac', 'onestype', 'skoroport_frac', 'twostype', 'zerostype']]

  0%|          | 0/33707 [00:00<?, ?it/s]

In [47]:
danf_test

,naliv_frac,naval_frac,onestype,skoroport_frac,twostype,zerostype
0,0.0,0.841121,0.000000,0.0,0.122066,0.380282
1,0.0,0.898876,0.000000,0.0,0.089202,0.328638
2,0.0,0.560000,0.000000,0.0,0.220657,0.131455
3,0.0,0.440000,0.000000,0.0,0.206573,0.145540
4,0.0,0.635514,0.037559,0.0,0.286385,0.178404
...,...,...,...,...,...,...
33702,0.0,0.740000,0.000000,0.0,0.126761,0.342723
33703,0.0,0.709677,0.028475,0.0,0.208818,0.199326
33704,0.0,1.000000,0.000000,0.0,0.000000,0.347418
33705,0.0,1.000000,0.000000,0.0,0.107981,0.389671


#### Добавляем фичи из таблици wag_params

In [48]:
wag_params = pd.read_parquet('wag_params.parquet').convert_dtypes()

cols_to_drop = ['srok_sl', 'date_build', 'date_iskl']

wag_params = wag_params.drop(columns=cols_to_drop)

cat_feat = wag_params.columns[1:].values

params = targs.merge(wag_params, on = "wagnum", how = "left")[cat_feat].copy()

for p in cat_feat:
    params[p] = params[p].astype('category')

In [49]:
params_test = test.merge(wag_params, on = "wagnum", how = "left")[cat_feat].copy()

for p in cat_feat:
    params_test[p] = params_test[p].astype('category')

In [ ]:
# mf_train = pd.concat([mff, ef, eff, nerep, params, own_feat, trems_counts, danf, lastreptab], axis = 1)
# mf_test = pd.concat([mff_test, ef_test, eff_test, nerep_test, params_test, own_feat_test, trems_counts_test, danf_test, lastreptab_test], axis = 1)

In [ ]:
# test = pd.read_csv('target/y_predict.csv')
# mf_train = pd.concat([targs[["wagnum", "month"]], mf_train], axis = 1)
# mf_test = pd.concat([test[["wagnum", "month"]], mf_test], axis = 1)

In [ ]:
# mf_train

#### Далее мы строим финальное решение

#### В качестве алгоритма машинного обучения мы использовали бустинги из библиотеки LGBMClassifier, в финальном решение мы используем ансамбль из 7 бустингом с разными random_state, усредняя ответы для них

#### Подбор гиперпараметров происходит в следующей секции

In [ ]:
!pip install lightgbm

  Obtaining dependency information for lightgbm from https://files.pythonhosted.org/packages/b8/9d/1ce80cee7c5ef60f2fcc7e9fa97f29f7a8de3dc5a08922b3b2f1e9106481/lightgbm-4.1.0-py3-none-manylinux_2_28_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 5.5 MB/s eta 0:00:0000:0100:01


In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
y_pred_day = []
for i in range(7):
    model_day = LGBMClassifier(is_unbalance=True, num_threads = 8, force_row_wise=True, **{'n_estimators': 10000,
     'learning_rate': 0.013404981239009836 / 2,
     'min_split_gain': 0.03842333239584914,
     'max_depth': -1,
     'max_cat_threshold': 32,
     'cat_l2': 68.60685058667403,
     'random_state': i})

    model_day.fit(mf_train, targs.target_day)
    y_pred_day.append(model_day.predict(mf_test))

y_pred_day = np.concatenate([i.reshape(-1,1) for i in y_pred_day], axis=1)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 3892, number of negative: 233669
[LightGBM] [Info] Total Bins 7293
[LightGBM] [Info] Number of data points in the train set: 237561, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.016383 -> initscore=-4.094982
[LightGBM] [Info] Start training from score -4.094982


KeyboardInterrupt: 

In [ ]:
new_ans = y_pred_day.sum(axis=1).copy()
new_ans[y_pred_day.sum(axis=1).copy() > 3] = 1
new_ans[y_pred_day.sum(axis=1).copy() <= 3] = 0
y_pred_day = new_ans

In [ ]:
y_pred_month = []
for i in range(7):
    model_month = LGBMClassifier(is_unbalance=True, num_threads = 8, force_row_wise=True, **{'n_estimators': 10000,
     'learning_rate': 0.024922405732354058 / 2,
     'min_split_gain': 0.03126564532038549,
     'max_depth': 10,
     'max_cat_threshold': 128,
     'cat_l2': 55.69886937210401,
     'random_state': i})

    model_month.fit(mf_train, targs.target_month)
    y_pred_month.append(model_month.predict(mf_test))

y_pred_month = np.concatenate([i.reshape(-1,1) for i in y_pred_month], axis=1)

[LightGBM] [Warning] num_threads is set=8, n_jobs=-1 will be ignored. Current value: num_threads=8
[LightGBM] [Warning] num_threads is set=8, n_jobs=-1 will be ignored. Current value: num_threads=8
[LightGBM] [Warning] num_threads is set=8, n_jobs=-1 will be ignored. Current value: num_threads=8
[LightGBM] [Warning] num_threads is set=8, n_jobs=-1 will be ignored. Current value: num_threads=8


In [ ]:
new_ans = y_pred_month.sum(axis=1).copy()
new_ans[y_pred_month.sum(axis=1).copy() > 3] = 1
new_ans[y_pred_month.sum(axis=1).copy() <= 3] = 0
y_pred_month = new_ans

In [ ]:
submit = pd.read_csv('target/y_predict_submit_example.csv')
submit.target_month = y_pred_month
submit.target_day = y_pred_day
submit.to_csv('submit3.csv', index=False)

#### Здесь мы перебирали гиперпараметры с помощью optuna
#### В качестве валидационной выборки мы использовали данные за февраль

In [ ]:
mf = pd.concat([mff, ef, eff, nerep, params, own_feat, trems_counts, danf, lastreptab], axis = 1)

In [ ]:
feat = pd.concat([targs[["index", "wagnum", "month"]], mf], axis = 1)

In [ ]:
data_mask = feat.month.apply(lambda x:int(x[5:7])) != 8

In [ ]:
train_mask = (feat.month.apply(lambda x:int(x[5:7]))!= 8)&(feat.month.apply(lambda x:int(x[5:7]))!= 2)
test_mask = feat.month.apply(lambda x:int(x[5:7])) == 2

In [ ]:
X_train = mf[train_mask]
y_train = targs[train_mask].target_month

In [ ]:
X_test = mf[test_mask]
y_test = targs[test_mask].target_month

In [ ]:
X = mf[data_mask]
y = targs[data_mask].target_month.values

In [ ]:
import optuna
import numpy as np
#from optuna.integration import LightGBMPruningCallback
from sklearn.model_selection import StratifiedKFold

def objective(trial):
    param_grid = {
        # "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [500]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        #"num_leaves": trial.suggest_int("num_leaves", 20, 300, step=20),
        #"min_child_samples": trial.suggest_int("min_child_samples", 5, 100, step=5),
        #"lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        #"lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_split_gain": trial.suggest_float("min_split_gain", 0, 0.1),
        #"subsample": trial.suggest_float(
        #    "subsample", 0.2, 0.95, step=0.1
        #),
        #"subsample_freq": trial.suggest_categorical("subsample_freq", [1]),
        #"colsample_bytree": trial.suggest_float(
        #    "colsample_bytree", 0.2, 0.95, step=0.1
        #),
        "max_depth": trial.suggest_categorical("max_depth", [-1, 3, 4, 5, 6, 8, 10]),
        "max_cat_threshold": trial.suggest_categorical("max_cat_threshold", [8, 16, 32, 64, 128]),
        "cat_l2": trial.suggest_float("cat_l2", 5.0, 100.0)

    }


    lgbm = LGBMClassifier(is_unbalance=True, num_threads = 6, force_row_wise=True, **param_grid)
    cvr = []

    for ti, di in StratifiedKFold(n_splits=5, shuffle=True).split(X, y):
        Xt = X.iloc[ti]
        #print(ti)
        yt = y[ti]

        Xd = X.iloc[di]
        yd = y[di]

        lgbm.fit(
            Xt,
            yt
        )


        #lgbm.fit(Xt, yt, sample_weight = sample_weights[ti])

        ypred = lgbm.predict(Xd)


        cvr.append(f1_score(yd, ypred))

        #print(confusion_matrix(yd, [int(x) if x>=0 else 0 for x in np.round(ypred)]))

    return np.mean(cvr)


study = optuna.create_study(direction="maximize", study_name="LGBM Classifier")
study.optimize(objective, n_trials=100)

[I 2023-11-12 07:54:56,033] A new study created in memory with name: LGBM Classifier


[LightGBM] [Warning] num_threads is set=6, n_jobs=-1 will be ignored. Current value: num_threads=6
[LightGBM] [Warning] num_threads is set=6, n_jobs=-1 will be ignored. Current value: num_threads=6
[LightGBM] [Warning] num_threads is set=6, n_jobs=-1 will be ignored. Current value: num_threads=6
[LightGBM] [Warning] num_threads is set=6, n_jobs=-1 will be ignored. Current value: num_threads=6
[LightGBM] [Warning] num_threads is set=6, n_jobs=-1 will be ignored. Current value: num_threads=6


[I 2023-11-12 07:55:15,866] Trial 0 finished with value: 0.6860678544480908 and parameters: {'n_estimators': 500, 'learning_rate': 0.2424515371972734, 'min_split_gain': 0.02862180166588827, 'max_depth': 8, 'max_cat_threshold': 32, 'cat_l2': 51.395900406164415}. Best is trial 0 with value: 0.6860678544480908.


[LightGBM] [Warning] num_threads is set=6, n_jobs=-1 will be ignored. Current value: num_threads=6
[LightGBM] [Warning] num_threads is set=6, n_jobs=-1 will be ignored. Current value: num_threads=6
[LightGBM] [Warning] num_threads is set=6, n_jobs=-1 will be ignored. Current value: num_threads=6
[LightGBM] [Warning] num_threads is set=6, n_jobs=-1 will be ignored. Current value: num_threads=6
[LightGBM] [Warning] num_threads is set=6, n_jobs=-1 will be ignored. Current value: num_threads=6


[I 2023-11-12 07:55:34,723] Trial 1 finished with value: 0.5841128933778891 and parameters: {'n_estimators': 500, 'learning_rate': 0.021264367707282423, 'min_split_gain': 0.05720465530652167, 'max_depth': 5, 'max_cat_threshold': 64, 'cat_l2': 56.976424177648845}. Best is trial 0 with value: 0.6860678544480908.


...

[LightGBM] [Warning] num_threads is set=6, n_jobs=-1 will be ignored. Current value: num_threads=6
[LightGBM] [Warning] num_threads is set=6, n_jobs=-1 will be ignored. Current value: num_threads=6
[LightGBM] [Warning] num_threads is set=6, n_jobs=-1 will be ignored. Current value: num_threads=6
[LightGBM] [Warning] num_threads is set=6, n_jobs=-1 will be ignored. Current value: num_threads=6
[LightGBM] [Warning] num_threads is set=6, n_jobs=-1 will be ignored. Current value: num_threads=6


[I 2023-11-12 08:24:03,182] Trial 88 finished with value: 0.6783074742883776 and parameters: {'n_estimators': 500, 'learning_rate': 0.23778628667105395, 'min_split_gain': 0.03863277641214161, 'max_depth': 5, 'max_cat_threshold': 32, 'cat_l2': 60.726914347843845}. Best is trial 31 with value: 0.6908604431032936.


[LightGBM] [Warning] num_threads is set=6, n_jobs=-1 will be ignored. Current value: num_threads=6
[LightGBM] [Warning] num_threads is set=6, n_jobs=-1 will be ignored. Current value: num_threads=6
[LightGBM] [Warning] num_threads is set=6, n_jobs=-1 will be ignored. Current value: num_threads=6


[W 2023-11-12 08:24:15,286] Trial 89 failed with parameters: {'n_estimators': 500, 'learning_rate': 0.27093410663966183, 'min_split_gain': 0.035938970211483005, 'max_depth': 10, 'max_cat_threshold': 32, 'cat_l2': 40.637649698069765} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Егор\AppData\Local\Temp\ipykernel_14824\4251613687.py", line 33, in objective
    for ti, di in StratifiedKFold(n_splits=5, shuffle=True).split(X, y):
  File "C:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py", line 352, in split
    for train, test in super().split(X, y, groups):
  File "C:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py", line 87, in split
    test_index = indices[test_index]
KeyboardInterrupt
[W 2023-11-12 08:24:15,309] Trial 89 failed with value None.


KeyboardInterrupt: 

In [ ]:
study.best_params

{'n_estimators': 500,
 'learning_rate': 0.1653879638699648,
 'num_leaves': 60,
 'max_depth': 8,
 'max_cat_threshold': 32,
 'cat_l2': 25.2041278553189}

In [ ]:
lgbm = LGBMClassifier(is_unbalance=True, num_threads = 8, force_row_wise=True, seed = 77, **{'n_estimators': 500,
 'learning_rate': 0.13404981239009836,
 'min_split_gain': 0.03842333239584914,
 'max_depth': -1,
 'max_cat_threshold': 32,
 'cat_l2': 68.60685058667403})

lgbm.fit(X_train, y_train)

LGBMClassifier(cat_l2=68.60685058667403, force_row_wise=True, is_unbalance=True,
               learning_rate=0.13404981239009836, max_cat_threshold=32,
               min_split_gain=0.03842333239584914, n_estimators=500,
               num_threads=8, seed=77)

In [ ]:
y_pred = lgbm.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred)

0.5083333333333333

In [ ]:
sorted(list(zip(X_train.columns, lgbm.feature_importances_)), key=lambda x: x[1])[::-1]

[('probeg', 85000),
 ('time_before_srok_sl', 73727),
 ('age', 70465),
 ('under_load_percent', 66132),
 ('last_month_std', 62623),
 ('under_load_dist', 53323),
 ('time_before_remont', 49907),
 ('nearrepair', 41690),
 ('under_load_days', 39959),
 ('last_month_run', 36222),
 ('last_month_mean', 32189),
 ('cur_st_send', 28207),
 ('num_small_repairs', 13870),
 ('cur_st_dest', 10195),
 ('tormoz', 6178),
 ('kod_remont', 5781),
 ('kuzov', 5037),
 ('tara', 3113),
 ('ownertype', 2773),
 ('manage_type', 2388),
 ('rod_id', 1572),
 ('stat_remont', 1437),
 ('kod_vrab_tr_rems', 1173),
 ('model', 1081),
 ('zavod_build', 455),
 ('cnsi_probeg_kr', 204),
 ('tipvozd', 155),
 ('tippogl', 134),
 ('gruz', 10),
 ('norma_km', 0),
 ('telega', 0),
 ('cnsi_probeg_dr', 0),
 ('cnsi_volumek', 0),
 ('cnsi_gruz_capacity', 0)]

## Catboost

In [50]:
mf = pd.concat([mff, ef, eff, nerep, params, own_feat, trems_counts, danf, lastreptab], axis = 1)

In [51]:
feat = pd.concat([targs[["index", "wagnum", "month"]], mf], axis = 1)

In [52]:
data_mask = feat.month.apply(lambda x:int(x[5:7])) != 8

In [53]:
train_mask = (feat.month.apply(lambda x:int(x[5:7]))!= 8)&(feat.month.apply(lambda x:int(x[5:7]))!= 2)
test_mask = feat.month.apply(lambda x:int(x[5:7])) == 2

In [54]:
X_train = mf[train_mask]
y_train_month = targs[train_mask].target_month

In [55]:
X_train = mf[train_mask]
y_train_day = targs[train_mask].target_day

In [56]:
X_test = mf[test_mask]
y_test_month = targs[test_mask].target_month

In [57]:
X_test = mf[test_mask]
y_test_day = targs[test_mask].target_day

In [58]:
X = mf[data_mask]
y = targs[data_mask].target_month.values

In [59]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 10.1 MB/s eta 0:00:00


In [60]:
from catboost import CatBoostClassifier

In [61]:
cat_features = list(X_train.columns[X_train.dtypes == "category"])

In [62]:
catb_X_train = X_train.copy()
catb_X_test = X_test.copy()
catb_X = X.copy()

In [63]:
catb_X_train[cat_features] = catb_X_train[cat_features].astype(str)
catb_X_test[cat_features] = catb_X_test[cat_features].astype(str)

In [64]:
catb_X_train.fillna(0, inplace=True)
catb_X_test.fillna(0, inplace=True)

In [65]:
catb_X[cat_features] = catb_X[cat_features].astype(str)
catb_X.fillna(0, inplace=True)

### baseline catboost experiments

In [ ]:
catb_month = CatBoostClassifier(
    random_seed=42,
    # logging_level='Silent',
    cat_features=cat_features
)

In [ ]:
catb_month = CatBoostClassifier(
    random_seed=42,
    # logging_level='Silent',
    cat_features=cat_features,
    iterations=2000
)

In [ ]:
catb_month.fit(catb_X_train, y_train_month)

Learning rate set to 0.048883
0:	learn: 0.6011814	total: 274ms	remaining: 9m 6s
1:	learn: 0.5153351	total: 537ms	remaining: 8m 56s
2:	learn: 0.4194135	total: 761ms	remaining: 8m 26s
3:	learn: 0.3682201	total: 983ms	remaining: 8m 10s
4:	learn: 0.3190892	total: 1.19s	remaining: 7m 56s
5:	learn: 0.2715417	total: 1.43s	remaining: 7m 55s
6:	learn: 0.2377817	total: 1.67s	remaining: 7m 54s
7:	learn: 0.2123681	total: 1.91s	remaining: 7m 54s
8:	learn: 0.1913430	total: 2.13s	remaining: 7m 50s
9:	learn: 0.1732805	total: 2.35s	remaining: 7m 47s
10:	learn: 0.1595290	total: 2.56s	remaining: 7m 43s
11:	learn: 0.1489424	total: 2.8s	remaining: 7m 43s
12:	learn: 0.1387695	total: 3.04s	remaining: 7m 44s
13:	learn: 0.1292576	total: 3.29s	remaining: 7m 46s
14:	learn: 0.1214030	total: 3.51s	remaining: 7m 44s
15:	learn: 0.1140616	total: 3.76s	remaining: 7m 46s
16:	learn: 0.1103868	total: 4.01s	remaining: 7m 47s
17:	learn: 0.1078010	total: 4.27s	remaining: 7m 49s
18:	learn: 0.1036532	total: 4.52s	remaining: 7

In [ ]:
y_pred_month = catb_month.predict(catb_X_test)

In [ ]:
catb_day = CatBoostClassifier(
    random_seed=42,
    # logging_level='Silent',
    cat_features=cat_features
)

In [ ]:
catb_day = CatBoostClassifier(
    random_seed=42,
    # logging_level='Silent',
    cat_features=cat_features,
    iterations=2000
)

In [ ]:
catb_day.fit(catb_X_train, y_train_day)

Learning rate set to 0.048883
0:	learn: 0.5965150	total: 265ms	remaining: 8m 48s
1:	learn: 0.5177695	total: 559ms	remaining: 9m 18s
2:	learn: 0.4392791	total: 887ms	remaining: 9m 50s
3:	learn: 0.3719697	total: 1.2s	remaining: 9m 59s
4:	learn: 0.3270918	total: 1.44s	remaining: 9m 33s
5:	learn: 0.2907382	total: 1.49s	remaining: 8m 14s
6:	learn: 0.2500304	total: 1.63s	remaining: 7m 43s
7:	learn: 0.2254513	total: 1.73s	remaining: 7m 9s
8:	learn: 0.2046625	total: 1.87s	remaining: 6m 53s
9:	learn: 0.1783955	total: 2.19s	remaining: 7m 16s
10:	learn: 0.1530206	total: 2.48s	remaining: 7m 28s
11:	learn: 0.1349009	total: 2.79s	remaining: 7m 42s
12:	learn: 0.1183806	total: 3.07s	remaining: 7m 48s
13:	learn: 0.1051824	total: 3.37s	remaining: 7m 58s
14:	learn: 0.0918105	total: 3.71s	remaining: 8m 11s
15:	learn: 0.0803581	total: 4.09s	remaining: 8m 27s
16:	learn: 0.0723626	total: 4.44s	remaining: 8m 38s
17:	learn: 0.0667739	total: 4.77s	remaining: 8m 45s
18:	learn: 0.0620271	total: 5.06s	remaining: 8

In [ ]:
y_pred_day = catb_day.predict(catb_X_test)

In [ ]:
from sklearn.metrics import f1_score

#iterations=2000
month_score = f1_score(y_test_month, y_pred_month)
day_score = f1_score(y_test_day, y_pred_day)
print(month_score, day_score, (month_score + day_score) / 2)

0.6860734369831293 0.6174920969441517 0.6517827669636405


In [ ]:
from sklearn.metrics import f1_score

#iterations=1000
month_score = f1_score(y_test_month, y_pred_month)
day_score = f1_score(y_test_day, y_pred_day)
print(month_score, day_score, (month_score + day_score) / 2)

0.6811062979006998 0.6217948717948718 0.6514505848477858


In [ ]:
sorted(zip(catb_month.feature_names_, catb_month.feature_importances_), key=lambda x: x[1], reverse=True)

[('probeg', 45.33480980852727),
 ('lastrepair', 9.37554934821155),
 ('nearrepair', 7.385409474220373),
 ('time_before_remont', 6.228876711720449),
 ('last_month_run', 2.305777164208808),
 ('zerostype', 2.0616416465615037),
 ('twostype', 1.9750704456338364),
 ('cur_st_dest', 1.9372384974987922),
 ('num_small_repairs', 1.8034677023175214),
 ('kod_vrab_tr_rems', 1.7979521876480316),
 ('last_month_mean', 1.6279337296030763),
 ('under_load_dist', 1.5579660394543808),
 ('under_load_percent', 1.488675726037661),
 ('last_month_std', 1.4818001624609136),
 ('age', 1.4701704989806743),
 ('cur_st_send', 1.23861585599143),
 ('onestype', 1.0984570750880966),
 ('naval_frac', 1.0642694009867997),
 ('tara', 0.9859413873823673),
 ('time_before_srok_sl', 0.9354143658289628),
 ('under_load_days', 0.6987092316790435),
 ('model', 0.49794540383500224),
 ('cnsi_volumek', 0.49784662801642526),
 ('rod_id', 0.44929627833971375),
 ('stat_remont', 0.4415230444400417),
 ('ownertype', 0.41886537674675245),
 ('cnsi_p

In [ ]:
sorted(zip(catb_day.feature_names_, catb_day.feature_importances_), key=lambda x: x[1], reverse=True)

[('probeg', 46.79317548047109),
 ('lastrepair', 6.354820090844153),
 ('time_before_remont', 5.487152573029469),
 ('nearrepair', 5.052763243168936),
 ('cur_st_dest', 2.8244014708815866),
 ('kod_vrab_tr_rems', 2.307616236512592),
 ('num_small_repairs', 2.205607923656645),
 ('time_before_srok_sl', 2.0699246025920073),
 ('gruz', 1.9150575370258525),
 ('age', 1.875782976373751),
 ('last_month_mean', 1.7694147747031626),
 ('cur_st_send', 1.738081489187057),
 ('cnt_repairs', 1.7200631892755225),
 ('last_month_std', 1.686373748872904),
 ('naval_frac', 1.6295292628293647),
 ('zerostype', 1.6218905876222565),
 ('last_month_run', 1.441502062765582),
 ('under_load_dist', 1.335873108172304),
 ('twostype', 1.318250624438879),
 ('tippogl', 1.2406001942926974),
 ('tara', 1.2125908042209148),
 ('under_load_percent', 1.1586473535475397),
 ('under_load_days', 1.0547076519696126),
 ('onestype', 0.8893163410538465),
 ('tipvozd', 0.5344316655719921),
 ('zavod_build', 0.5175084097702054),
 ('kod_remont', 0.3

In [ ]:
y_pred_day = []
for i in tqdm(range(7)):
    catb_day = CatBoostClassifier(
        random_seed=i,
        logging_level='Silent',
        cat_features=cat_features
    )

    catb_day.fit(catb_X_train, y_train_day)
    y_pred_day.append(catb_day.predict(catb_X_test))

y_pred_day = np.concatenate([i.reshape(-1,1) for i in y_pred_day], axis=1)

  0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
new_ans = y_pred_day.sum(axis=1).copy()
new_ans[y_pred_day.sum(axis=1).copy() > 3] = 1
new_ans[y_pred_day.sum(axis=1).copy() <= 3] = 0
y_pred_day = new_ans

In [ ]:
y_pred_month = []
for i in tqdm(range(7)):
    catb_month = CatBoostClassifier(
        random_seed=i,
        logging_level='Silent',
        cat_features=cat_features
    )

    catb_month.fit(catb_X_train, y_train_month)
    y_pred_month.append(catb_month.predict(catb_X_test))

y_pred_month = np.concatenate([i.reshape(-1,1) for i in y_pred_month], axis=1)

  0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
new_ans = y_pred_month.sum(axis=1).copy()
new_ans[y_pred_month.sum(axis=1).copy() > 3] = 1
new_ans[y_pred_month.sum(axis=1).copy() <= 3] = 0
y_pred_month = new_ans

In [ ]:
from sklearn.metrics import f1_score

month_score = f1_score(y_test_month, y_pred_month)
day_score = f1_score(y_test_day, y_pred_day)
print(month_score, day_score, (month_score + day_score) / 2)

0.686300463269358 0.6252676659528907 0.6557840646111244


### GPU Catboost

In [ ]:
catb_month = CatBoostClassifier(
    random_seed=42,
    # logging_level='Silent',
    cat_features=cat_features,
    task_type="GPU",
    # devices='0:1',
)

In [ ]:
catb_day = CatBoostClassifier(
    random_seed=42,
    # logging_level='Silent',
    cat_features=cat_features,
    task_type="GPU",
    # devices='0:1',
)

In [ ]:
catb_month.fit(catb_X_train, y_train_month)

Learning rate set to 0.025417
0:	learn: 0.6192703	total: 168ms	remaining: 2m 48s
1:	learn: 0.5793986	total: 342ms	remaining: 2m 50s
2:	learn: 0.5384200	total: 495ms	remaining: 2m 44s
3:	learn: 0.4950049	total: 671ms	remaining: 2m 47s
4:	learn: 0.4551544	total: 836ms	remaining: 2m 46s
5:	learn: 0.4207312	total: 1s	remaining: 2m 46s
6:	learn: 0.3808647	total: 1.17s	remaining: 2m 46s
7:	learn: 0.3555508	total: 1.27s	remaining: 2m 37s
8:	learn: 0.3244011	total: 1.39s	remaining: 2m 32s
9:	learn: 0.2981329	total: 1.48s	remaining: 2m 26s
10:	learn: 0.2749494	total: 1.65s	remaining: 2m 27s
11:	learn: 0.2549741	total: 1.8s	remaining: 2m 28s
12:	learn: 0.2360077	total: 1.98s	remaining: 2m 29s
13:	learn: 0.2197196	total: 2.04s	remaining: 2m 23s
14:	learn: 0.2050033	total: 2.09s	remaining: 2m 17s
15:	learn: 0.1921903	total: 2.14s	remaining: 2m 11s
16:	learn: 0.1825380	total: 2.19s	remaining: 2m 6s
17:	learn: 0.1735735	total: 2.25s	remaining: 2m 2s
18:	learn: 0.1646906	total: 2.4s	remaining: 2m 3s


In [ ]:
y_pred_month = catb_month.predict(catb_X_test)

In [ ]:
catb_day.fit(catb_X_train, y_train_day)

Learning rate set to 0.025417
0:	learn: 0.6329429	total: 98.9ms	remaining: 1m 38s
1:	learn: 0.5719045	total: 171ms	remaining: 1m 25s
2:	learn: 0.5315468	total: 242ms	remaining: 1m 20s
3:	learn: 0.4878811	total: 362ms	remaining: 1m 30s
4:	learn: 0.4404377	total: 472ms	remaining: 1m 33s
5:	learn: 0.4063692	total: 580ms	remaining: 1m 36s
6:	learn: 0.3737462	total: 682ms	remaining: 1m 36s
7:	learn: 0.3390601	total: 810ms	remaining: 1m 40s
8:	learn: 0.3135273	total: 916ms	remaining: 1m 40s
9:	learn: 0.2928323	total: 963ms	remaining: 1m 35s
10:	learn: 0.2754716	total: 1.05s	remaining: 1m 34s
11:	learn: 0.2581575	total: 1.15s	remaining: 1m 34s
12:	learn: 0.2381049	total: 1.24s	remaining: 1m 33s
13:	learn: 0.2220320	total: 1.32s	remaining: 1m 33s
14:	learn: 0.2081949	total: 1.37s	remaining: 1m 29s
15:	learn: 0.1916214	total: 1.42s	remaining: 1m 27s
16:	learn: 0.1705475	total: 1.46s	remaining: 1m 24s
17:	learn: 0.1521973	total: 1.52s	remaining: 1m 22s
18:	learn: 0.1437988	total: 1.57s	remaining

In [ ]:
y_pred_day = catb_day.predict(catb_X_test)

In [ ]:
from sklearn.metrics import f1_score

month_score = f1_score(y_test_month, y_pred_month)
day_score = f1_score(y_test_day, y_pred_day)
print(month_score, day_score, (month_score + day_score) / 2)

0.6868150116240451 0.5925925925925926 0.6397038021083188


### Tuning catboost

Неплохой буст в качестве получается когда увеличиваем one_hot_max_size

In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.1 MB/s eta 0:00:00


In [ ]:
import optuna
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from catboost import Pool

train_pool = Pool(catb_X, label = y, cat_features=cat_features)

def objective(trial):
    param_grid = {
        "iterations": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10, log=True),
        "leaf_estimation_iterations": trial.suggest_int("leaf_estimation_iterations", 1, 10)
    }

    clf = CatBoostClassifier(
        random_seed=42,
        logging_level='Silent',
        cat_features=cat_features,
        task_type="GPU",
    )
    cvr = []

    for ti, di in StratifiedKFold(n_splits=3, shuffle=True).split(catb_X, y):  # n_splits = 3
        # Xt = catb_X.iloc[ti]
        # yt = y[ti]
        # Xd = catb_X.iloc[di]
        yd = y[di]

        train_slice = train_pool.slice(ti)
        test_slice = train_pool.slice(di)

        clf.fit(train_slice)

        ypred = clf.predict(test_slice)


        cvr.append(f1_score(yd, ypred))

        #print(confusion_matrix(yd, [int(x) if x>=0 else 0 for x in np.round(ypred)]))

    return np.mean(cvr)


study = optuna.create_study(direction="maximize", study_name="CatBoost Classifier")
study.optimize(objective, n_trials=100)

[I 2023-12-21 17:11:30,189] A new study created in memory with name: CatBoost Classifier
[I 2023-12-21 17:14:36,980] Trial 0 finished with value: 0.692567866334788 and parameters: {'learning_rate': 0.002460722273510353, 'depth': 4, 'subsample': 0.5860982196724367, 'colsample_bylevel': 0.42709495534950254, 'min_data_in_leaf': 9, 'bagging_temperature': 0.4574759375937508, 'l2_leaf_reg': 2.5434957754823215, 'leaf_estimation_iterations': 8}. Best is trial 0 with value: 0.692567866334788.
[I 2023-12-21 17:17:22,970] Trial 1 finished with value: 0.6968506719870708 and parameters: {'learning_rate': 0.005541531642456082, 'depth': 10, 'subsample': 0.6969070794391691, 'colsample_bylevel': 0.5684402268013333, 'min_data_in_leaf': 56, 'bagging_temperature': 0.9807995974613467, 'l2_leaf_reg': 2.5107763763633906, 'leaf_estimation_iterations': 3}. Best is trial 1 with value: 0.6968506719870708.
[I 2023-12-21 17:20:04,649] Trial 2 finished with value: 0.6913895577920037 and parameters: {'learning_rate'

In [66]:
best_params_month = {'learning_rate': 0.005541531642456082,
                     'depth': 10,
                     'subsample': 0.6969070794391691,
                     'colsample_bylevel': 0.5684402268013333,
                     'min_data_in_leaf': 56,
                     'bagging_temperature': 0.9807995974613467,
                     'l2_leaf_reg': 2.5107763763633906,
                     'leaf_estimation_iterations': 3}

catb_month = CatBoostClassifier(
    random_seed=42,
    # logging_level='Silent',
    cat_features=cat_features,
    **best_params_month
)

In [68]:
catb_month.fit(catb_X_train, y_train_month)

0:	learn: 0.6839723	total: 374ms	remaining: 6m 13s
1:	learn: 0.6761409	total: 495ms	remaining: 4m 7s
2:	learn: 0.6674617	total: 629ms	remaining: 3m 29s
3:	learn: 0.6586353	total: 1.03s	remaining: 4m 16s
4:	learn: 0.6498633	total: 1.52s	remaining: 5m 2s
5:	learn: 0.6411713	total: 1.92s	remaining: 5m 18s
6:	learn: 0.6346979	total: 1.96s	remaining: 4m 37s
7:	learn: 0.6276522	total: 2.04s	remaining: 4m 12s
8:	learn: 0.6197270	total: 2.59s	remaining: 4m 45s
9:	learn: 0.6115545	total: 3.31s	remaining: 5m 28s
10:	learn: 0.6037313	total: 3.64s	remaining: 5m 27s
11:	learn: 0.5958507	total: 4.01s	remaining: 5m 29s
12:	learn: 0.5899970	total: 4.1s	remaining: 5m 11s
13:	learn: 0.5827007	total: 4.41s	remaining: 5m 10s
14:	learn: 0.5747904	total: 4.99s	remaining: 5m 27s
15:	learn: 0.5671936	total: 5.52s	remaining: 5m 39s
16:	learn: 0.5596938	total: 6.14s	remaining: 5m 54s
17:	learn: 0.5521574	total: 6.71s	remaining: 6m 6s
18:	learn: 0.5452167	total: 6.99s	remaining: 6m 1s
19:	learn: 0.5400406	total: